### **Площадь здания на OSM**

In [ ]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import pandas as pd

In [1]:
data = pd.read_excel('jakarta_fitness_itog.xlsx')

In [2]:
len(data)

2154

In [ ]:
def get_nearest_building_area(lat, lon):
    point = Point(lon, lat)
    try:
        # Загружаем здания в радиусе 30 метров от точки
        buildings = ox.features_from_point((lat, lon), tags={'building': True}, dist=350)

        if not buildings.empty:
            # Преобразуем в проекцию в метрах
            buildings = buildings.to_crs(epsg=3857)
            point_proj = gpd.GeoSeries([point], crs='EPSG:4326').to_crs(epsg=3857).iloc[0]

            # Вычисляем расстояние до каждого здания в метрах
            buildings['distance'] = buildings.geometry.distance(point_proj)

            # Сортируем по расстоянию и берем ближайшее
            nearest_building = buildings.sort_values('distance').iloc[0]

            area = nearest_building.geometry.area
            intersects = nearest_building.geometry.intersects(point_proj)

            return area, 1 if intersects else 0
        else:
            return None, 0
    except Exception as e:
        print(f"Error for coordinates ({lat}, {lon}): {e}")
        return None, 0

# Обработка данных
results = []
for index, row in data.iterrows():
    area, intersect = get_nearest_building_area(row['latitude'], row['longitude'])
    results.append({
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'area': area,
        'intersect': intersect
    })
    if index%100==0:
      print(index)
      pd.DataFrame(results).to_excel('jakarta_builds_area_osm.xlsx')


result_df = pd.DataFrame(results)

In [6]:
result_df.to_excel('jakarta_builds_area_osm.xlsx', index = False)

In [7]:
data['area_building_osm'] = result_df['area']
data['intersect_building_osm'] = result_df['intersect']

In [8]:
data.to_excel('jakarta_fitness_itog.xlsx', index = False)